# Project 1 — Fine-Tuning a Pre-Trained Transformer Model

---
Submitted by: Akshita Sharma

Roll No.: 2301730278

In [ ]:
!pip install transformers datasets accelerate evaluate scikit-learn -q

In [ ]:
import pandas as pd
from datasets import load_dataset
import numpy as np
import re

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Create dummy email data
data = {
    'subject': [
        'Meeting Tomorrow',
        'Quick Question',
        'Project Update',
        'Lunch Plans',
        'Urgent: Review Needed',
        'Weekly Report',
        'Feedback Request',
        'Invoice Attached',
        'Team Outing',
        'System Maintenance',
        'New Feature Launch',
        'Budget Approval',
        'Training Session',
        'Holiday Notice',
        'Congratulations!',
        'Follow Up',
        'Reminder: Deadline',
        'Thank You',
        'Survey Request',
        'Meeting Minutes'
    ],
    'body': [
        'Hi, just wanted to confirm our meeting scheduled for tomorrow at 10 AM.',
        'Can you please share the latest version of the document?',
        'The project is progressing well. We are on track to meet the deadline.',
        'Would you like to grab lunch today around 1 PM?',
        'Please review the attached document as soon as possible.',
        'Please find the weekly report attached for your review.',
        'I would appreciate your feedback on the proposal.',
        'Please find the invoice for last month attached.',
        'We are planning a team outing next Friday. Please confirm your availability.',
        'System maintenance is scheduled for this weekend.',
        'Excited to announce the launch of our new feature!',
        'The budget proposal has been approved by management.',
        'Training session on new tools will be held next week.',
        'Office will be closed next Monday for the holiday.',
        'Congratulations on your promotion! Well deserved.',
        'Following up on my previous email regarding the contract.',
        'Reminder: Project deadline is approaching this Friday.',
        'Thank you for your assistance with the presentation.',
        'Please take a moment to complete our customer survey.',
        'Attached are the minutes from yesterday\'s meeting.'
    ],
    'label': [0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1]
}

# Create a DataFrame and convert to Dataset
df = pd.DataFrame(data)

# Convert to Hugging Face Dataset
full_dataset = Dataset.from_pandas(df)

# Split into train/test (80/20)
dataset = full_dataset.train_test_split(test_size=0.2, seed=42)

# Split test into val/test (50/50)
temp = dataset["test"].train_test_split(test_size=0.5, seed=42)
dataset["val"] = temp["train"]
dataset["test"] = temp["test"]

# Display the dataset structure
dataset

DatasetDict({
    train: Dataset({
        features: ['subject', 'body', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['subject', 'body', 'label'],
        num_rows: 2
    })
    val: Dataset({
        features: ['subject', 'body', 'label'],
        num_rows: 2
    })
})

In [ ]:
model_name = "distilbert-base-uncased"
# OR
# model_name = "bert-base-uncased"
# model_name = "roberta-base"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="spam_detector",
    eval_strategy="epoch",    # new name (evaluation_strategy deprecated)
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True
)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="spam_detector",
    eval_strategy="epoch",    # new name (evaluation_strategy deprecated)
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True
)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1": f1.compute(predictions=preds, references=labels, average="binary")["f1"],
    }

In [ ]:
import torch

def predict_email(text):
    device = next(model.parameters()).device  # detect CPU or GPU
    inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return "SPAM" if pred == 1 else "NOT SPAM"

In [ ]:
print(predict_email("Congratulations! You won $1000 gift card."))

NOT SPAM
